Imports

In [34]:
# [CELL 01-01] Imports (UPDATED for raw XuetangX JSON -> Parquet -> DuckDB)

import os
import sys
import json
import math
import time
from pathlib import Path
from datetime import datetime

import yaml

try:
    import pandas as pd
except Exception as e:
    pd = None
    print("⚠️ pandas not available:", e)

def log(msg: str):
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[01] {ts} | {msg}")


Bootstrap

In [35]:
# [CELL 01-02] Bootstrap: locate repo root reliably (Windows-safe)

CWD = Path.cwd().resolve()
log(f"Initial CWD: {CWD}")

def find_repo_root(start: Path) -> Path:
    """
    Search upward for repo root.
    Priority: PROJECT_STATE.md (most specific).
    Fallback: .git folder.
    """
    for p in [start, *start.parents]:
        if (p / "PROJECT_STATE.md").exists():
            return p.resolve()

    for p in [start, *start.parents]:
        if (p / ".git").exists():
            return p.resolve()

    # Last resort: don't guess; just use current folder
    return start.resolve()

REPO_ROOT = find_repo_root(CWD)
log(f"Detected REPO_ROOT: {REPO_ROOT}")

# Move execution context to repo root (so relative paths behave consistently)
os.chdir(REPO_ROOT)
log(f"CWD now: {Path.cwd().resolve()}")

# Add repo root + src to import path
SRC_DIR = REPO_ROOT / "src"
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))
if SRC_DIR.exists() and str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

# Validation checks
checks = {
    "src/": (REPO_ROOT / "src").exists(),
    "notebooks/": (REPO_ROOT / "notebooks").exists(),
    "PROJECT_STATE.md": (REPO_ROOT / "PROJECT_STATE.md").exists(),
    "src/configs/project.yaml": (REPO_ROOT / "src" / "configs" / "project.yaml").exists(),
}

log("Validation checks:")
for name, exists in checks.items():
    status = "✅" if exists else "❌"
    print(f"  {status} {name}")

if not checks["PROJECT_STATE.md"]:
    raise FileNotFoundError("PROJECT_STATE.md not found in detected repo root — wrong working directory?")

if not checks["src/configs/project.yaml"]:
    raise FileNotFoundError("src/configs/project.yaml not found — please ensure the config exists.")


[01] 2025-12-28 23:10:27 | Initial CWD: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta
[01] 2025-12-28 23:10:27 | Detected REPO_ROOT: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta
[01] 2025-12-28 23:10:27 | CWD now: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta
[01] 2025-12-28 23:10:27 | Validation checks:
  ✅ src/
  ✅ notebooks/
  ✅ PROJECT_STATE.md
  ✅ src/configs/project.yaml


Config loader (src/configs/project.yaml)

In [36]:
# [CELL 01-03] Load project config (YAML) 

CFG_PATH = REPO_ROOT / "src" / "configs" / "project.yaml"
log(f"Loading config: {CFG_PATH}")

with open(CFG_PATH, "r", encoding="utf-8") as f:
    CFG = yaml.safe_load(f) or {}

log("Config loaded.")
log(f"Top-level keys: {sorted(list(CFG.keys()))}")

def cfg_get(*path, default=None):
    """
    Safe nested getter: cfg_get('data','raw_dir')
    """
    cur = CFG
    for k in path:
        if not isinstance(cur, dict) or k not in cur:
            return default
        cur = cur[k]
    return cur

# Try common conventions without assuming which one you used
RAW_DIR_CANDIDATES = [
    cfg_get("paths", "raw_dir"),
    cfg_get("data", "raw_dir"),
    cfg_get("data", "raw"),
    cfg_get("paths", "data_raw"),
]

RAW_DIR_VALUE = next((x for x in RAW_DIR_CANDIDATES if isinstance(x, str) and x.strip()), None)

if RAW_DIR_VALUE is None:
    log("⚠️ Could not find raw dir in config using common keys.")
    log("Searched keys: paths.raw_dir, data.raw_dir, data.raw, paths.data_raw")
    RAW_DIR = None
else:
    RAW_DIR = (REPO_ROOT / RAW_DIR_VALUE).resolve()
    log(f"RAW_DIR from config: {RAW_DIR_VALUE}  ->  {RAW_DIR}")
    log(f"RAW_DIR exists: {RAW_DIR.exists()}")

# Try common conventions for "source dataset subdir"
SOURCE_SUBDIR_CANDIDATES = [
    cfg_get("datasets", "source_mooc", "subdir"),
    cfg_get("datasets", "source", "subdir"),
    cfg_get("data", "source_mooc_subdir"),
    cfg_get("source_dataset", "subdir"),
]

SOURCE_SUBDIR = next((x for x in SOURCE_SUBDIR_CANDIDATES if isinstance(x, str) and x.strip()), None)
if SOURCE_SUBDIR is None:
    log("ℹ️ No explicit source dataset subdir found in config (that’s okay).")
    SOURCE_DIR = None if RAW_DIR is None else RAW_DIR
else:
    SOURCE_DIR = None if RAW_DIR is None else (RAW_DIR / SOURCE_SUBDIR).resolve()
    log(f"SOURCE_SUBDIR from config: {SOURCE_SUBDIR}")
    log(f"SOURCE_DIR resolved: {SOURCE_DIR}")
    log(f"SOURCE_DIR exists: {SOURCE_DIR.exists()}")


[01] 2025-12-28 23:10:41 | Loading config: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\src\configs\project.yaml
[01] 2025-12-28 23:10:41 | Config loaded.
[01] 2025-12-28 23:10:41 | Top-level keys: ['paths', 'project', 'repro', 'training']
[01] 2025-12-28 23:10:41 | RAW_DIR from config: data/raw  ->  D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\data\raw
[01] 2025-12-28 23:10:41 | RAW_DIR exists: True
[01] 2025-12-28 23:10:41 | ℹ️ No explicit source dataset subdir found in config (that’s okay).


data/raw inventory + file-size summary

In [38]:
# [CELL 01-04] Inventory data/raw and confirm XuetangX raw folder exists

if RAW_DIR is None:
    raise RuntimeError("RAW_DIR is None. Config loader did not resolve raw dir.")

if not RAW_DIR.exists():
    raise FileNotFoundError(f"RAW_DIR does not exist: {RAW_DIR}")

log(f"Scanning RAW_DIR: {RAW_DIR}")

def human_bytes(n: int) -> str:
    units = ["B", "KB", "MB", "GB", "TB"]
    f = float(n)
    for u in units:
        if f < 1024 or u == units[-1]:
            return f"{f:.2f} {u}"
        f /= 1024

files = []
for p in RAW_DIR.rglob("*"):
    if p.is_file():
        try:
            size = p.stat().st_size
        except OSError:
            size = None
        files.append((p, size))

log(f"Found files: {len(files)}")
total = sum(s for _, s in files if isinstance(s, int))
log(f"Total size: {human_bytes(total)}")

subdirs = sorted([d for d in RAW_DIR.iterdir() if d.is_dir()])
log(f"Top-level subdirs under RAW_DIR ({len(subdirs)}): {[d.name for d in subdirs]}")

# Confirm xuetangx folder specifically
XUETANGX_DIR = RAW_DIR / "xuetangx"
log(f"Expecting XuetangX raw folder: {XUETANGX_DIR}")
log(f"Exists: {XUETANGX_DIR.exists()}")

if not XUETANGX_DIR.exists():
    raise FileNotFoundError("Expected data/raw/xuetangx folder not found. Please place raw JSON files there.")


[01] 2025-12-28 23:11:47 | Scanning RAW_DIR: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\data\raw
[01] 2025-12-28 23:11:47 | Found files: 11
[01] 2025-12-28 23:11:47 | Total size: 29.46 GB
[01] 2025-12-28 23:11:47 | Top-level subdirs under RAW_DIR (2): ['mars', 'xuetangx']
[01] 2025-12-28 23:11:47 | Expecting XuetangX raw folder: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\data\raw\xuetangx
[01] 2025-12-28 23:11:47 | Exists: True


Explicitly select the “source MOOC dataset”

In [39]:
# [CELL 01-05] Select which raw subdir is the source dataset

SOURCE_DATASET_SUBDIR = "xuetangx"  # explicitly set

SOURCE_DIR = (RAW_DIR / SOURCE_DATASET_SUBDIR).resolve()
log(f"SOURCE_DIR: {SOURCE_DIR}")
log(f"SOURCE_DIR exists: {SOURCE_DIR.exists()}")

if not SOURCE_DIR.exists():
    raise FileNotFoundError(f"SOURCE_DIR does not exist: {SOURCE_DIR}")

json_files = sorted(SOURCE_DIR.glob("*raw_user_activity*.json"))
log(f"Raw XuetangX JSON files found: {len(json_files)}")
for p in json_files:
    print(f"  - {p.name} | {human_bytes(p.stat().st_size)}")

if not json_files:
    raise FileNotFoundError("No *raw_user_activity*.json files found in data/raw/xuetangx/")


[01] 2025-12-28 23:12:14 | SOURCE_DIR: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\data\raw\xuetangx
[01] 2025-12-28 23:12:14 | SOURCE_DIR exists: True
[01] 2025-12-28 23:12:14 | Raw XuetangX JSON files found: 6
  - 20150801-20151101-raw_user_activity.json | 3.65 GB
  - 20151101-20160201-raw_user_activity.json | 4.97 GB
  - 20160201-20160501-raw_user_activity.json | 5.33 GB
  - 20160501-20160801-raw_user_activity.json | 5.17 GB
  - 20160801-20170201-raw_user_activity.json | 5.53 GB
  - 20170201-20170801-raw_user_activity.json | 4.79 GB


Quick HEAD/TAIL check (confirm JSON array + structure)

In [40]:
# [CELL 01-06] Confirm JSON structure via HEAD/TAIL bytes

def head_tail_bytes(path: Path, n=220):
    size = path.stat().st_size
    with open(path, "rb") as f:
        head = f.read(n)
        f.seek(max(0, size - n))
        tail = f.read(n)
    return head, tail

p = json_files[0]
log(f"Inspecting: {p.name}")
head, tail = head_tail_bytes(p)

print("HEAD:", head[:220])
print("TAIL:", tail[:220])

# sanity: should start with '[' and end with ']'
if not head.lstrip().startswith(b"["):
    log("⚠️ File does not appear to start with '['; structure might differ.")
if b"]" not in tail:
    log("⚠️ File does not appear to end with ']'; structure might differ.")


[01] 2025-12-28 23:12:55 | Inspecting: 20150801-20151101-raw_user_activity.json
HEAD: b'[\n  [\n    "course-v1:BIT+PHY1701601+2015_T2", \n    {\n      "1482755": {\n        "2939ce48edf106610df7abbdf70b6862": [\n          [\n            "click_about", \n            "2015-10-26T11:49:16"\n          ], \n          [\n  '
TAIL: b'        ], \n          [\n            "pause_video", \n            "2015-10-20T21:53:24"\n          ], \n          [\n            "close_courseware", \n            "2015-10-20T21:53:30"\n          ]\n        ]\n      }\n    }\n  ]\n]'


Install/Import dependencies (ijson, pyarrow, duckdb)

In [41]:
# [CELL 01-07] Ensure required libs are available: ijson, pyarrow, duckdb

import importlib
import subprocess

def ensure(pkg: str, import_name: str = None):
    name = import_name or pkg
    try:
        return importlib.import_module(name)
    except Exception:
        log(f"Installing {pkg} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
        return importlib.import_module(name)

ijson = ensure("ijson")
duckdb = ensure("duckdb")
pa = ensure("pyarrow", "pyarrow")
pq = ensure("pyarrow", "pyarrow.parquet")

log("All dependencies ready: ijson, pyarrow, duckdb")


[01] 2025-12-28 23:13:32 | Installing duckdb ...
[01] 2025-12-28 23:16:25 | All dependencies ready: ijson, pyarrow, duckdb


Stream-sample and summarize (no full load)

This confirms: actions, time range, and typical nesting.

In [42]:
# [CELL 01-08] Stream-sample raw JSON (K course blocks) and summarize actions/time range

from collections import Counter

def iter_course_blocks(path: Path):
    # top-level JSON is an array; each item is expected: [course_id, user_map]
    with open(path, "rb") as f:
        for item in ijson.items(f, "item"):
            yield item

def sample_summary(path: Path, K=5, max_users_per_course=50, max_events_per_object=500):
    action_counts = Counter()
    total_events = 0
    total_users = 0
    total_objects = 0
    min_ts = None
    max_ts = None

    seen_courses = 0
    for blk in iter_course_blocks(path):
        if not (isinstance(blk, list) and len(blk) == 2):
            continue

        course_id, user_map = blk[0], blk[1]
        if not (isinstance(course_id, str) and isinstance(user_map, dict)):
            continue

        seen_courses += 1

        for ui, (user_id, obj_map) in enumerate(user_map.items()):
            if ui >= max_users_per_course:
                break
            total_users += 1

            if not isinstance(obj_map, dict):
                continue

            for obj_id, evs in obj_map.items():
                total_objects += 1
                if not isinstance(evs, list):
                    continue

                for ev in evs[:max_events_per_object]:
                    if not (isinstance(ev, list) and len(ev) == 2):
                        continue
                    action, ts = ev[0], ev[1]
                    action_counts[str(action)] += 1
                    total_events += 1

                    dt = pd.to_datetime(ts, errors="coerce")
                    if pd.notna(dt):
                        if min_ts is None or dt < min_ts:
                            min_ts = dt
                        if max_ts is None or dt > max_ts:
                            max_ts = dt

        if seen_courses >= K:
            break

    print(f"\nSampled file: {path.name}")
    print("Sampled courses:", seen_courses)
    print("Sampled users:", total_users)
    print("Sampled objects:", total_objects)
    print("Sampled events:", total_events)
    print("Sample time range:", min_ts, "->", max_ts)
    print("\nTop actions:")
    for a, c in action_counts.most_common(20):
        print(f"  {a:25s} {c}")

# sample the first file
sample_summary(json_files[0], K=5)



Sampled file: 20150801-20151101-raw_user_activity.json
Sampled courses: 5
Sampled users: 250
Sampled objects: 434
Sampled events: 27745
Sample time range: 2015-08-01 00:56:17 -> 2015-10-31 23:29:23

Top actions:
  play_video                3980
  pause_video               3355
  load_video                3279
  stop_video                3209
  problem_get               3192
  problem_check             2060
  seek_video                1955
  close_courseware          1899
  problem_check_correct     1428
  click_courseware          1412
  problem_check_incorrect   626
  click_info                445
  click_about               315
  click_progress            281
  problem_save              221
  click_forum               76
  create_comment            6
  create_thread             3
  reset_problem             3


One-time conversion: Stream parse → Parquet shards

This creates a clean event table:
course_id, user_id, object_id, action, ts

In [43]:
# [CELL 01-09] Stream parse -> Parquet shards (one-time conversion)

OUT_DIR = (REPO_ROOT / "data" / "processed" / "xuetangx_events_parquet").resolve()
OUT_DIR.mkdir(parents=True, exist_ok=True)
log(f"OUT_DIR: {OUT_DIR}")

def write_shard(rows, shard_path: Path):
    df = pd.DataFrame(rows, columns=["course_id", "user_id", "object_id", "action", "ts"])
    df["ts"] = pd.to_datetime(df["ts"], errors="coerce")
    df = df.dropna(subset=["course_id", "user_id", "action", "ts"])
    table = pa.Table.from_pandas(df, preserve_index=False)
    pq.write_table(table, shard_path)
    return len(df)

def convert_file_to_parquet_shards(path: Path, shard_rows=1_000_000, max_blocks=None):
    shard_idx = 0
    buf = []
    total_written = 0
    blocks = 0

    t0 = time.time()
    with open(path, "rb") as f:
        for blk in ijson.items(f, "item"):
            blocks += 1
            if max_blocks and blocks > max_blocks:
                break

            if not (isinstance(blk, list) and len(blk) == 2):
                continue
            course_id, user_map = blk[0], blk[1]
            if not (isinstance(course_id, str) and isinstance(user_map, dict)):
                continue

            for user_id, obj_map in user_map.items():
                if not isinstance(obj_map, dict):
                    continue
                for object_id, evs in obj_map.items():
                    if not isinstance(evs, list):
                        continue
                    for ev in evs:
                        if not (isinstance(ev, list) and len(ev) == 2):
                            continue
                        action, ts = ev[0], ev[1]
                        buf.append([course_id, str(user_id), str(object_id), str(action), ts])

                        if len(buf) >= shard_rows:
                            shard_path = OUT_DIR / f"{path.stem}_shard_{shard_idx:04d}.parquet"
                            n = write_shard(buf, shard_path)
                            total_written += n
                            buf = []
                            shard_idx += 1

                            elapsed = time.time() - t0
                            log(f"{path.name} | shards={shard_idx} | written={total_written:,} | elapsed={elapsed/60:.1f}m")

    if buf:
        shard_path = OUT_DIR / f"{path.stem}_shard_{shard_idx:04d}.parquet"
        n = write_shard(buf, shard_path)
        total_written += n
        shard_idx += 1

    elapsed = time.time() - t0
    log(f"DONE {path.name} | shards={shard_idx} | total_written={total_written:,} | elapsed={elapsed/60:.1f}m")
    return total_written, shard_idx

# Recommended: run one file first to validate, then run all.
# 1) Validate on first file:
# convert_file_to_parquet_shards(json_files[0], shard_rows=1_000_000)

# 2) After validation, convert all:
totals = []
for p in json_files:
    total_written, n_shards = convert_file_to_parquet_shards(p, shard_rows=1_000_000)
    totals.append((p.name, total_written, n_shards))

print("\nConversion summary:")
for name, rows_written, shards in totals:
    print(f"- {name}: rows={rows_written:,} shards={shards}")


[01] 2025-12-28 23:30:38 | OUT_DIR: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\data\processed\xuetangx_events_parquet
[01] 2025-12-28 23:30:55 | 20150801-20151101-raw_user_activity.json | shards=1 | written=1,000,000 | elapsed=0.3m
[01] 2025-12-28 23:30:57 | 20150801-20151101-raw_user_activity.json | shards=2 | written=2,000,000 | elapsed=0.3m
[01] 2025-12-28 23:31:02 | 20150801-20151101-raw_user_activity.json | shards=3 | written=3,000,000 | elapsed=0.4m
[01] 2025-12-28 23:31:09 | 20150801-20151101-raw_user_activity.json | shards=4 | written=4,000,000 | elapsed=0.5m
[01] 2025-12-28 23:31:11 | 20150801-20151101-raw_user_activity.json | shards=5 | written=5,000,000 | elapsed=0.5m
[01] 2025-12-28 23:31:14 | 20150801-20151101-raw_user_activity.json | shards=6 | written=6,000,000 | elapsed=0.6m
[01] 2025-12-28 23:31:19 | 20150801-20151101-raw_user_activity.json | shards=7 | written=7,000,000 | elapsed=0.7m
[01] 2025-12-28 23:31:23 | 20150801-20151101-raw_user_activity.json | shards=

DuckDB EDA on Parquet (fast)

In [44]:
# [CELL 01-10] DuckDB EDA on Parquet shards

PARQUET_GLOB = str(OUT_DIR / "*.parquet")
log(f"Reading Parquet via DuckDB: {PARQUET_GLOB}")

con = duckdb.connect(database=":memory:")

con.execute(f"""
CREATE VIEW events AS
SELECT * FROM read_parquet('{PARQUET_GLOB}');
""")

# Basic counts
print("Rows:", con.execute("SELECT COUNT(*) FROM events").fetchone()[0])
print("Users:", con.execute("SELECT COUNT(DISTINCT user_id) FROM events").fetchone()[0])
print("Courses:", con.execute("SELECT COUNT(DISTINCT course_id) FROM events").fetchone()[0])
print("Objects:", con.execute("SELECT COUNT(DISTINCT object_id) FROM events").fetchone()[0])

# Time range
print("Time range:", con.execute("SELECT MIN(ts), MAX(ts) FROM events").fetchone())

# Top actions
df_top_actions = con.execute("""
SELECT action, COUNT(*) AS n
FROM events
GROUP BY action
ORDER BY n DESC
LIMIT 30
""").df()
display(df_top_actions)

# Per-user event count distribution (quick)
df_user_dist = con.execute("""
SELECT
  approx_quantile(cnt, [0.5, 0.9, 0.99]) AS quantiles
FROM (
  SELECT user_id, COUNT(*) cnt
  FROM events
  GROUP BY user_id
);
""").df()
display(df_user_dist)


[01] 2025-12-28 23:55:49 | Reading Parquet via DuckDB: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\data\processed\xuetangx_events_parquet\*.parquet


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Rows: 351452376


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Users: 772887
Courses: 1629


...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Time range: (datetime.datetime(2015, 7, 31, 23, 59, 14), datetime.datetime(2017, 7, 31, 23, 59, 10))


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,action,n
0,stop_video,64090652
1,pause_video,57140305
2,click_courseware,44017420
3,play_video,43325389
4,seek_video,27074548
5,problem_get,24958594
6,load_video,24354867
7,close_courseware,20505645
8,click_about,9709480
9,problem_check,9422514


,quantiles
0,"[33, 1017, 6331]"


Decide interaction “item_id” + export interactions (optional)

In [45]:
# [CELL 01-11] Build interactions table (choose item granularity)

ITEM_MODE = "course"  # "course" or "object"

if ITEM_MODE == "course":
    item_col = "course_id"
elif ITEM_MODE == "object":
    item_col = "object_id"
else:
    raise ValueError("ITEM_MODE must be 'course' or 'object'")

# Keep only actions you consider as interactions (optional)
# Leave as None to keep all.
KEEP_ACTIONS = None  # e.g. {"click_courseware","play_video","problem_check","click_forum"}

where = ""
if KEEP_ACTIONS:
    actions_list = ",".join([f"'{a}'" for a in sorted(KEEP_ACTIONS)])
    where = f"WHERE action IN ({actions_list})"

inter_path = (REPO_ROOT / "data" / "processed" / f"xuetangx_interactions_{ITEM_MODE}.parquet").resolve()

con.execute(f"""
COPY (
  SELECT
    user_id,
    {item_col} AS item_id,
    ts,
    action,
    course_id
  FROM events
  {where}
) TO '{str(inter_path)}' (FORMAT PARQUET);
""")

log(f"Wrote interactions parquet: {inter_path}")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[01] 2025-12-29 00:00:47 | Wrote interactions parquet: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\data\processed\xuetangx_interactions_course.parquet


Reports folder + run metadata + save core summary tables

In [52]:
# [CELL 01-12] Reports: folder + metadata + save core summary tables

import json
from datetime import datetime
from pathlib import Path

REPORT_DIR = (REPO_ROOT / "reports" / "01_eda_source_mooc").resolve()
RUN_TAG = datetime.now().strftime("%Y%m%d_%H%M%S")
OUT = (REPORT_DIR / RUN_TAG)
OUT.mkdir(parents=True, exist_ok=True)
log(f"REPORT OUT: {OUT}")

# Save a small "run metadata" json
meta = {
    "run_tag": RUN_TAG,
    "source_dir": str(SOURCE_DIR),
    "events_parquet_glob": str(OUT_DIR / "*.parquet"),
    "filtered_interactions_parquet": str(REPO_ROOT / "data" / "processed" / "xuetangx_interactions_course_filtered.parquet"),
}

with open(OUT / "run_meta.json", "w", encoding="utf-8") as f:
    json.dump(meta, f, indent=2)

# Core EDA tables (small) from DuckDB
df_counts = con.execute("""
SELECT
  COUNT(*) AS n_events,
  COUNT(DISTINCT user_id) AS n_users,
  COUNT(DISTINCT course_id) AS n_courses,
  COUNT(DISTINCT object_id) AS n_objects,
  MIN(ts) AS min_ts,
  MAX(ts) AS max_ts
FROM events;
""").df()

df_top_actions = con.execute("""
SELECT action, COUNT(*) AS n
FROM events
GROUP BY action
ORDER BY n DESC
LIMIT 50;
""").df()

df_user_quant = con.execute("""
SELECT approx_quantile(cnt, [0.5, 0.9, 0.99]) AS q_events_per_user
FROM (
  SELECT user_id, COUNT(*) cnt
  FROM events
  GROUP BY user_id
);
""").df()

display(df_counts)
display(df_top_actions.head(20))
display(df_user_quant)

# Save to CSV for the report folder
df_counts.to_csv(OUT / "counts_overview.csv", index=False)
df_top_actions.to_csv(OUT / "top_actions.csv", index=False)
df_user_quant.to_csv(OUT / "user_event_quantiles.csv", index=False)

log("Saved core EDA tables to reports folder.")


[01] 2025-12-29 01:30:02 | REPORT OUT: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\reports\01_eda_source_mooc\20251229_013002


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,n_events,n_users,n_courses,n_objects,min_ts,max_ts
0,351452376,772887,1629,2937678,2015-07-31 23:59:14,2017-07-31 23:59:10


,action,n
0,stop_video,64090652
1,pause_video,57140305
2,click_courseware,44017420
3,play_video,43325389
4,seek_video,27074548
5,problem_get,24958594
6,load_video,24354867
7,close_courseware,20505645
8,click_about,9709480
9,problem_check,9422514


,q_events_per_user
0,"[33, 1022, 6352]"


[01] 2025-12-29 01:30:57 | Saved core EDA tables to reports folder.


Plot: Top actions bar chart (save PNG)

In [ ]:
# [CELL 01-13] Plot: Top actions (save)

import matplotlib.pyplot as plt

topN = 20
dfp = df_top_actions.head(topN).copy()

plt.figure(figsize=(10, 6))
plt.barh(dfp["action"][::-1], dfp["n"][::-1])
plt.xlabel("Event count")
plt.title(f"Top {topN} actions (XuetangX raw events)")
plt.tight_layout()

png = OUT / f"plot_top_{topN}_actions.png"
plt.savefig(png, dpi=200)
plt.show()

log(f"Saved: {png}")


Plot: Events over time (daily) from DuckDB (fast) + save

In [ ]:
# [CELL 01-14] Plot: Daily event volume (fast via DuckDB aggregation)

df_daily = con.execute("""
SELECT
  DATE_TRUNC('day', ts) AS day,
  COUNT(*) AS n
FROM events
GROUP BY day
ORDER BY day;
""").df()

display(df_daily.head())

plt.figure(figsize=(12, 4))
plt.plot(df_daily["day"], df_daily["n"])
plt.xlabel("Day")
plt.ylabel("Events")
plt.title("Daily event volume (raw XuetangX)")
plt.tight_layout()

png = OUT / "plot_daily_event_volume.png"
plt.savefig(png, dpi=200)
plt.show()

df_daily.to_csv(OUT / "daily_event_volume.csv", index=False)
log(f"Saved: {png}")


Plot: Course popularity (top 50 courses by events) + save

In [ ]:
# [CELL 01-15] Plot: Course popularity (top courses by event count)

df_course_top = con.execute("""
SELECT course_id, COUNT(*) AS n
FROM events
GROUP BY course_id
ORDER BY n DESC
LIMIT 50;
""").df()

display(df_course_top.head())

plt.figure(figsize=(10, 8))
plt.barh(df_course_top["course_id"][::-1], df_course_top["n"][::-1])
plt.xlabel("Event count")
plt.title("Top 50 courses by event count")
plt.tight_layout()

png = OUT / "plot_top_50_courses.png"
plt.savefig(png, dpi=200)
plt.show()

df_course_top.to_csv(OUT / "top_50_courses.csv", index=False)
log(f"Saved: {png}")


Plot: User activity distribution (log-scale histogram) + save

In [ ]:
# [CELL 01-16] Plot: User activity distribution (histogram of events per user)

# Pull a manageable sample of per-user counts (DuckDB computes it; result is large but still feasible)
# If memory is a concern, add a LIMIT sample; but start with full aggregation.
df_user_counts = con.execute("""
SELECT user_id, COUNT(*) AS n
FROM events
GROUP BY user_id;
""").df()

display(df_user_counts.head())

plt.figure(figsize=(10, 4))
plt.hist(df_user_counts["n"], bins=200, log=True)
plt.xlabel("Events per user")
plt.ylabel("Frequency (log)")
plt.title("User activity distribution (events per user)")
plt.tight_layout()

png = OUT / "plot_user_events_hist_log.png"
plt.savefig(png, dpi=200)
plt.show()

# Save quantiles (small) + optionally a downsample
df_user_counts["n"].describe(percentiles=[0.5,0.9,0.99]).to_csv(OUT / "user_events_describe.csv")
log(f"Saved: {png}")


Write a short Markdown report (auto) into reports folder

In [ ]:
# [CELL 01-17] Create a short Markdown summary report

md = []
md.append(f"# 01 — EDA Source MOOC (XuetangX) — {RUN_TAG}\n")
md.append("## Inputs\n")
md.append(f"- Raw folder: `{SOURCE_DIR}`\n")
md.append(f"- Parquet shards: `{OUT_DIR}/*.parquet`\n")
md.append("\n## Overview counts\n")
md.append(df_counts.to_markdown(index=False))
md.append("\n\n## Top actions (first 20)\n")
md.append(df_top_actions.head(20).to_markdown(index=False))
md.append("\n\n## User event count quantiles\n")
md.append(df_user_quant.to_markdown(index=False))
md.append("\n\n## Saved figures\n")
md.append("- plot_top_20_actions.png\n")
md.append("- plot_daily_event_volume.png\n")
md.append("- plot_top_50_courses.png\n")
md.append("- plot_user_events_hist_log.png\n")

report_path = OUT / "report_01_eda_source_mooc.md"
report_path.write_text("\n".join(md), encoding="utf-8")
log(f"Wrote report: {report_path}")


Plot: Top actions (save PNG + CSV)

In [ ]:
# [CELL 01-14] Plot: Top actions

import matplotlib.pyplot as plt

df_top = con.execute("""
SELECT action, COUNT(*) AS n
FROM events
GROUP BY action
ORDER BY n DESC
LIMIT 20;
""").df()

df_top.to_csv(OUT / "top_actions_top20.csv", index=False)

plt.figure(figsize=(10, 6))
plt.barh(df_top["action"][::-1], df_top["n"][::-1])
plt.xlabel("Event count")
plt.title("Top 20 actions (XuetangX raw events)")
plt.tight_layout()

png = OUT / "plot_top20_actions.png"
plt.savefig(png, dpi=200)
plt.show()

log(f"Saved: {png}")


Plot: Daily event volume (save PNG + CSV)

In [ ]:
# [CELL 01-15] Plot: Daily event volume

df_daily = con.execute("""
SELECT DATE_TRUNC('day', ts) AS day, COUNT(*) AS n
FROM events
GROUP BY day
ORDER BY day;
""").df()

df_daily.to_csv(OUT / "daily_event_volume.csv", index=False)

plt.figure(figsize=(12, 4))
plt.plot(df_daily["day"], df_daily["n"])
plt.xlabel("Day")
plt.ylabel("Events")
plt.title("Daily event volume (XuetangX raw events)")
plt.tight_layout()

png = OUT / "plot_daily_event_volume.png"
plt.savefig(png, dpi=200)
plt.show()

log(f"Saved: {png}")


Plot: User activity distribution (binned in DuckDB, log-friendly)

In [ ]:
# [CELL 01-16] Plot: User activity distribution (log10 bins)

df_bins = con.execute("""
WITH u AS (
  SELECT user_id, COUNT(*) AS cnt
  FROM events
  GROUP BY user_id
),
b AS (
  SELECT
    CAST(FLOOR(LOG10(cnt)) AS INTEGER) AS log10_bin,
    COUNT(*) AS n_users
  FROM u
  GROUP BY log10_bin
)
SELECT * FROM b
ORDER BY log10_bin;
""").df()

df_bins.to_csv(OUT / "user_activity_log10_bins.csv", index=False)

plt.figure(figsize=(8, 4))
plt.bar(df_bins["log10_bin"], df_bins["n_users"])
plt.xlabel("log10(events per user) bin")
plt.ylabel("Number of users")
plt.title("User activity distribution (binned by log10)")
plt.tight_layout()

png = OUT / "plot_user_activity_log10_bins.png"
plt.savefig(png, dpi=200)
plt.show()

log(f"Saved: {png}")


Plot: Top courses by event count (save PNG + CSV)

In [ ]:
# [CELL 01-17] Plot: Top 20 courses by event count

df_courses = con.execute("""
SELECT course_id, COUNT(*) AS n
FROM events
GROUP BY course_id
ORDER BY n DESC
LIMIT 20;
""").df()

df_courses.to_csv(OUT / "top_courses_top20.csv", index=False)

plt.figure(figsize=(10, 7))
plt.barh(df_courses["course_id"][::-1], df_courses["n"][::-1])
plt.xlabel("Event count")
plt.title("Top 20 courses by event count")
plt.tight_layout()

png = OUT / "plot_top20_courses.png"
plt.savefig(png, dpi=200)
plt.show()

log(f"Saved: {png}")


Write a short Markdown report file (so Notebook 01 produces an artifact)

In [ ]:
# [CELL 01-18] Write Markdown report

df_counts = con.execute("""
SELECT
  COUNT(*) AS n_events,
  COUNT(DISTINCT user_id) AS n_users,
  COUNT(DISTINCT course_id) AS n_courses,
  COUNT(DISTINCT object_id) AS n_objects,
  MIN(ts) AS min_ts,
  MAX(ts) AS max_ts
FROM events;
""").df()

md = []
md.append(f"# 01 — EDA Source MOOC (XuetangX raw) — {RUN_TAG}\n")
md.append("## Overview\n")
md.append(df_counts.to_markdown(index=False))
md.append("\n\n## Artifacts\n")
md.append("- dataset_metadata.json (also saved under data/processed/...)\n")
md.append("- plot_top20_actions.png\n")
md.append("- plot_daily_event_volume.png\n")
md.append("- plot_user_activity_log10_bins.png\n")
md.append("- plot_top20_courses.png\n")

report_path = OUT / "report_01_eda_source_mooc.md"
report_path.write_text("\n".join(md), encoding="utf-8")
log(f"Wrote: {report_path}")


Generate related plots (Matplotlib) + save to reports/01_eda_source_mooc/<RUN_TAG>/

In [ ]:
# [CELL 01-12] Reports output folder (per-run)

import json
import platform
from datetime import datetime
from pathlib import Path

REPORT_DIR = (REPO_ROOT / "reports" / "01_eda_source_mooc").resolve()
RUN_TAG = datetime.now().strftime("%Y%m%d_%H%M%S")
OUT = REPORT_DIR / RUN_TAG
OUT.mkdir(parents=True, exist_ok=True)

log(f"REPORT OUT: {OUT}")


Save dataset metadata (reproducibility)

In [57]:
# [CELL 01-13] Save dataset metadata for reproducibility

import sys
import platform

# Raw files inventory
raw_files = []
for p in sorted(SOURCE_DIR.glob("*.json")):
    st = p.stat()
    raw_files.append({
        "name": p.name,
        "path": str(p),
        "size_bytes": int(st.st_size),
        "mtime": datetime.fromtimestamp(st.st_mtime).isoformat(timespec="seconds"),
    })

# Parquet shards inventory
parquet_files = sorted(OUT_DIR.glob("*.parquet"))
parquet_total_bytes = sum(p.stat().st_size for p in parquet_files)

# Core counts + time range + top actions + user quantiles from DuckDB
counts = con.execute("""
SELECT
  COUNT(*) AS n_events,
  COUNT(DISTINCT user_id) AS n_users,
  COUNT(DISTINCT course_id) AS n_courses,
  COUNT(DISTINCT object_id) AS n_objects,
  MIN(ts) AS min_ts,
  MAX(ts) AS max_ts
FROM events;
""").fetchone()

top_actions = con.execute("""
SELECT action, COUNT(*) AS n
FROM events
GROUP BY action
ORDER BY n DESC
LIMIT 30;
""").df().to_dict(orient="records")

user_q = con.execute("""
SELECT approx_quantile(cnt, [0.5, 0.9, 0.99]) AS q_events_per_user
FROM (SELECT user_id, COUNT(*) cnt FROM events GROUP BY user_id);
""").fetchone()[0]

meta = {
    "generated_at": datetime.now().isoformat(timespec="seconds"),
    "repo_root": str(REPO_ROOT),
    "source_dir": str(SOURCE_DIR),
    "raw_files": raw_files,
    "events_parquet_dir": str(OUT_DIR),
    "events_parquet_shards": len(parquet_files),
    "events_parquet_total_bytes": int(parquet_total_bytes),
    "duckdb_events_view": str(OUT_DIR / "*.parquet"),
    "counts": {
        "n_events": int(counts[0]),
        "n_users": int(counts[1]),
        "n_courses": int(counts[2]),
        "n_objects": int(counts[3]),
        "min_ts": str(counts[4]),
        "max_ts": str(counts[5]),
    },
    "top_actions": top_actions,
    "events_per_user_quantiles": user_q,
    "env": {
        "python": sys.version.split()[0],
        "platform": platform.platform(),
        "pandas": getattr(pd, "__version__", "unknown"),
        "duckdb": getattr(duckdb, "__version__", "unknown"),
        "pyarrow": getattr(pa, "__version__", "unknown"),
        "ijson": getattr(ijson, "__version__", "unknown"),
    },
    "notes": [
        "This notebook (01) covers EDA + JSON→Parquet conversion + DuckDB summaries only.",
        "Session-gap analysis and sessionization are planned for notebooks 04/05.",
    ],
}

# Save metadata to processed + reports
meta_proc_path = (OUT_DIR / "dataset_metadata.json")
meta_rep_path = (OUT / "dataset_metadata.json")

meta_proc_path.write_text(json.dumps(meta, indent=2), encoding="utf-8")
meta_rep_path.write_text(json.dumps(meta, indent=2), encoding="utf-8")

log(f"Saved metadata: {meta_proc_path}")
log(f"Saved metadata copy: {meta_rep_path}")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[01] 2025-12-29 01:51:20 | Saved metadata: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\data\processed\xuetangx_events_parquet\dataset_metadata.json
[01] 2025-12-29 01:51:20 | Saved metadata copy: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\reports\01_eda_source_mooc\20251229_013002\dataset_metadata.json


Data Quality Report

In [53]:
# [CELL 01-18] Data quality summary

print("="*70)
print("DATA QUALITY REPORT")
print("="*70)

# Missing values
df_missing = con.execute("""
SELECT
  SUM(CASE WHEN user_id IS NULL THEN 1 ELSE 0 END) AS null_users,
  SUM(CASE WHEN course_id IS NULL THEN 1 ELSE 0 END) AS null_courses,
  SUM(CASE WHEN ts IS NULL THEN 1 ELSE 0 END) AS null_timestamps,
  SUM(CASE WHEN action IS NULL THEN 1 ELSE 0 END) AS null_actions
FROM events;
""").df()

print("\nMissing values:")
display(df_missing)

# Temporal coverage
df_temporal = con.execute("""
SELECT
  DATE_TRUNC('month', ts) AS month,
  COUNT(*) AS events,
  COUNT(DISTINCT user_id) AS active_users
FROM events
GROUP BY month
ORDER BY month;
""").df()

print("\nTemporal distribution:")
display(df_temporal)

# Action diversity per user
df_action_div = con.execute("""
WITH user_actions AS (
  SELECT user_id, COUNT(DISTINCT action) AS n_distinct_actions
  FROM events
  GROUP BY user_id
)
SELECT
  approx_quantile(n_distinct_actions, [0.5, 0.9, 0.99]) AS q_action_diversity
FROM user_actions;
""").df()

print("\nAction diversity per user:")
display(df_action_div)

DATA QUALITY REPORT


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Missing values:


,null_users,null_courses,null_timestamps,null_actions
0,0.0,0.0,0.0,0.0


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Temporal distribution:


,month,events,active_users
0,2015-07-01,94,21
1,2015-08-01,5237094,28215
2,2015-09-01,11550102,61114
3,2015-10-01,27032075,75277
4,2015-11-01,26533170,77930
5,2015-12-01,19850262,75996
6,2016-01-01,13046764,61471
7,2016-02-01,10604181,54890
8,2016-03-01,26517475,103378
9,2016-04-01,26623544,93939


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Action diversity per user:


,q_action_diversity
0,"[7, 15, 18]"


Cold-Start Analysis

In [ ]:
# [CELL 01-19] Cold-start scenario analysis

print("="*70)
print("COLD-START READINESS ANALYSIS")
print("="*70)

# Users with <5 events (cold-start users)
df_coldstart = con.execute("""
WITH user_counts AS (
  SELECT user_id, COUNT(*) AS n_events
  FROM events
  GROUP BY user_id
)
SELECT
  SUM(CASE WHEN n_events < 5 THEN 1 ELSE 0 END) AS users_lt_5_events,
  SUM(CASE WHEN n_events < 10 THEN 1 ELSE 0 END) AS users_lt_10_events,
  SUM(CASE WHEN n_events < 20 THEN 1 ELSE 0 END) AS users_lt_20_events,
  COUNT(*) AS total_users
FROM user_counts;
""").df()

print("\nCold-start user distribution:")
display(df_coldstart)

# First-session characteristics
df_first_sess = con.execute("""
WITH first_sessions AS (
  SELECT user_id, sess_idx, COUNT(*) AS sess_len
  FROM sessioned
  WHERE sess_idx = 1
  GROUP BY user_id, sess_idx
)
SELECT
  approx_quantile(sess_len, [0.5, 0.9, 0.99]) AS q_first_session_length
FROM first_sessions;
""").df()

print("\nFirst session lengths:")
display(df_first_sess)

print("\n✅ Cold-start readiness:")
print("  - If >50% users have <20 events: GOOD for cold-start research")
print("  - First session P50 >5 events: Can build initial preferences")

Create filtered interaction parquet (course-level)

In [46]:
# [CELL 01-12] Filter to meaningful interaction actions + export interactions (course-level)

KEEP_ACTIONS = {
    "click_courseware",
    "load_video",
    "play_video",
    "problem_get",
    "problem_check",
    "click_info",
    "click_about",
    "click_progress",
    "click_forum",
}

actions_list = ",".join([f"'{a}'" for a in sorted(KEEP_ACTIONS)])
filtered_path = (REPO_ROOT / "data" / "processed" / "xuetangx_interactions_course_filtered.parquet").resolve()

con.execute(f"""
COPY (
  SELECT
    user_id,
    course_id AS item_id,
    ts,
    action
  FROM events
  WHERE action IN ({actions_list})
) TO '{str(filtered_path)}' (FORMAT PARQUET);
""")

log(f"Wrote filtered interactions parquet: {filtered_path}")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[01] 2025-12-29 00:03:42 | Wrote filtered interactions parquet: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\data\processed\xuetangx_interactions_course_filtered.parquet


Sessionization EDA (time-gap sessions on filtered interactions)

In [47]:
# [CELL 01-13] Sessionization EDA (time-gap sessions) using filtered interactions

gap_minutes = 30

con.execute(f"""
CREATE OR REPLACE VIEW inter AS
SELECT * FROM read_parquet('{str(filtered_path)}');
""")

df_stats = con.execute(f"""
WITH x AS (
  SELECT
    user_id,
    item_id,
    ts,
    LAG(ts) OVER (PARTITION BY user_id ORDER BY ts) AS prev_ts
  FROM inter
),
y AS (
  SELECT
    user_id,
    item_id,
    ts,
    CASE
      WHEN prev_ts IS NULL THEN 1
      WHEN ts - prev_ts > INTERVAL '{gap_minutes} minutes' THEN 1
      ELSE 0
    END AS new_sess
  FROM x
),
z AS (
  SELECT
    user_id,
    item_id,
    ts,
    SUM(new_sess) OVER (PARTITION BY user_id ORDER BY ts ROWS UNBOUNDED PRECEDING) AS sess_idx
  FROM y
),
sizes AS (
  SELECT user_id, sess_idx, COUNT(*) AS sess_len
  FROM z
  GROUP BY user_id, sess_idx
)
SELECT
  COUNT(*) AS n_sessions,
  AVG(sess_len) AS avg_len,
  approx_quantile(sess_len, [0.5, 0.9, 0.99]) AS q,
  MAX(sess_len) AS max_len,
  SUM(CASE WHEN sess_len >= 2 THEN 1 ELSE 0 END) AS sessions_ge_2,
  SUM(CASE WHEN sess_len >= 3 THEN 1 ELSE 0 END) AS sessions_ge_3
FROM sizes;
""").df()

display(df_stats)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,n_sessions,avg_len,q,max_len,sessions_ge_2,sessions_ge_3
0,6350920,26.424289,"[11, 65, 208]",131827,5488136.0,4961653.0


Sequence readiness EDA (per-user unique course counts)

In [48]:
# [CELL 01-14] Sequence readiness: user unique course counts (filtered)

df_user = con.execute("""
WITH u AS (
  SELECT user_id, COUNT(*) AS n_events, COUNT(DISTINCT item_id) AS n_items
  FROM inter
  GROUP BY user_id
)
SELECT
  approx_quantile(n_events, [0.5,0.9,0.99]) AS q_events,
  approx_quantile(n_items,  [0.5,0.9,0.99]) AS q_items
FROM u;
""").df()

display(df_user)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,q_events,q_items
0,"[22, 542, 2927]","[2, 8, 35]"


Inspect long-session outliers and decide caps

In [ ]:
# [CELL 01-15] Inspect session length outliers (top 20)

gap_minutes = 30

df_top = con.execute(f"""
WITH x AS (
  SELECT
    user_id,
    item_id,
    ts,
    LAG(ts) OVER (PARTITION BY user_id ORDER BY ts) AS prev_ts
  FROM inter
),
y AS (
  SELECT
    user_id,
    item_id,
    ts,
    CASE
      WHEN prev_ts IS NULL THEN 1
      WHEN ts - prev_ts > INTERVAL '{gap_minutes} minutes' THEN 1
      ELSE 0
    END AS new_sess
  FROM x
),
z AS (
  SELECT
    user_id,
    item_id,
    ts,
    SUM(new_sess) OVER (PARTITION BY user_id ORDER BY ts ROWS UNBOUNDED PRECEDING) AS sess_idx
  FROM y
),
sizes AS (
  SELECT user_id, sess_idx, COUNT(*) AS sess_len
  FROM z
  GROUP BY user_id, sess_idx
)
SELECT * FROM sizes
ORDER BY sess_len DESC
LIMIT 20;
""").df()

display(df_top)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,user_id,sess_idx,sess_len
0,720244,20.0,131827
1,1089016,24.0,64001
2,1901922,1.0,48775
3,504314,26.0,47244
4,510501,38.0,43696
5,691208,9.0,43345
6,25325,1.0,42071
7,486806,27.0,36692
8,1476553,58.0,30282
9,6822632,13.0,27301


Build session-level course sequences (dedupe course repeats within session)

In [50]:
# [CELL 01-16] Build session-level sequences (course-level), dedup consecutive repeats

gap_minutes = 30
MAX_EVENTS_PER_SESSION = 500

# Create a "sessioned events" view with session ids
con.execute(f"""
CREATE OR REPLACE VIEW sessioned AS
WITH x AS (
  SELECT
    user_id,
    item_id,
    ts,
    LAG(ts) OVER (PARTITION BY user_id ORDER BY ts) AS prev_ts
  FROM inter
),
y AS (
  SELECT
    user_id,
    item_id,
    ts,
    CASE
      WHEN prev_ts IS NULL THEN 1
      WHEN ts - prev_ts > INTERVAL '{gap_minutes} minutes' THEN 1
      ELSE 0
    END AS new_sess
  FROM x
),
z AS (
  SELECT
    user_id,
    item_id,
    ts,
    SUM(new_sess) OVER (PARTITION BY user_id ORDER BY ts ROWS UNBOUNDED PRECEDING) AS sess_idx
  FROM y
),
ranked AS (
  SELECT
    user_id,
    sess_idx,
    item_id,
    ts,
    ROW_NUMBER() OVER (PARTITION BY user_id, sess_idx ORDER BY ts) AS rn,
    LAG(item_id) OVER (PARTITION BY user_id, sess_idx ORDER BY ts) AS prev_item
  FROM z
),
trimmed AS (
  SELECT *
  FROM ranked
  WHERE rn <= {MAX_EVENTS_PER_SESSION}
),
dedup AS (
  SELECT *
  FROM trimmed
  WHERE prev_item IS NULL OR item_id <> prev_item
)
SELECT
  user_id,
  sess_idx,
  item_id,
  ts
FROM dedup;
""")

# Session lengths after trimming+dedup
df_len = con.execute("""
SELECT
  COUNT(*) AS n_events,
  COUNT(DISTINCT user_id || '-' || sess_idx) AS n_sessions,
  approx_quantile(cnt, [0.5,0.9,0.99]) AS q_len
FROM (
  SELECT user_id, sess_idx, COUNT(*) cnt
  FROM sessioned
  GROUP BY user_id, sess_idx
);
""").df()
display(df_len)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,n_events,n_sessions,q_len
0,6350920,6350920,"[1, 3, 12]"


Export final session dataset for modeling notebooks

In [51]:
# [CELL 01-17] Export sessioned sequences to Parquet

OUT_SESS = (REPO_ROOT / "data" / "processed" / "xuetangx_sessions_course.parquet").resolve()

con.execute(f"""
COPY (
  SELECT
    user_id,
    CAST(sess_idx AS INTEGER) AS session_idx,
    item_id,
    ts
  FROM sessioned
) TO '{str(OUT_SESS)}' (FORMAT PARQUET);
""")

log(f"Wrote: {OUT_SESS}")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[01] 2025-12-29 01:24:23 | Wrote: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\data\processed\xuetangx_sessions_course.parquet
